In [13]:
import tensorflow as tf

from keras.layers import Conv2D, DepthwiseConv2D, AveragePooling2D, BatchNormalization, Input, Dense, Flatten
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from keras.layers.advanced_activations import ReLU
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

In [14]:
# Config variables
# Data images are not of the same size
INPUT_SIZE = [224, 224]
BATCH = 32

In [15]:
# No data augmentation
train_val_image_data_generator = ImageDataGenerator(rescale=1.0/255,
                                            rotation_range=20,
                                            zoom_range=0.2,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.2,
                                            horizontal_flip=True,
                                            fill_mode="nearest",
                                            validation_split=0.2
                                            )

train_data = train_val_image_data_generator.flow_from_directory('./data/',
                                                     target_size = INPUT_SIZE,
                                                     batch_size=BATCH,
                                                     subset='training',
                                                     shuffle=True,
                                                     classes=['with_mask','without_mask']
                                                     )
validate_data = train_val_image_data_generator.flow_from_directory('./data/',
                                                     target_size = INPUT_SIZE,
                                                     batch_size=BATCH,
                                                     subset='validation',
                                                     shuffle=True,
                                                     classes=['with_mask','without_mask']
                                                     )

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


In [16]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

#stem of the model
input = Input(shape = (224,224,3))
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

# main part of the model
x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
x = mobilnet_block(x, filters = 128, strides = 1)
x = mobilnet_block(x, filters = 256, strides = 2)
x = mobilnet_block(x, filters = 256, strides = 1)
x = mobilnet_block(x, filters = 512, strides = 2)
for _ in range (5):
     x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 1024, strides = 2)
x = mobilnet_block(x, filters = 1024, strides = 1)
x = AveragePooling2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
x = Flatten()(x)
output = Dense (units = 2, activation = 'softmax')(x)
mobilenet = Model(inputs=input, outputs=output)
mobilenet.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_81 (Batc (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_81 (ReLU)              (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d_39 (Depthwi (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_82 (Batc (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_82 (ReLU)              (None, 112, 112, 32)      0   

In [52]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
mobilenet.compile(
  loss='binary_crossentropy',
  optimizer='SGD',
  metrics=['accuracy', precision, recall]
)

es = EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, mode='auto')

mobilenet_r = mobilenet.fit(
  train_data,
  validation_data=validate_data,
  epochs=25,
  steps_per_epoch=len(train_data),
  validation_steps=len(validate_data),
  callbacks=[es]
)

Epoch 1/25
142/189 [=====================>........] - ETA: 1:27 - loss: 0.7361 - accuracy: 0.5977 - precision: 0.5925 - recall: 0.6388

d:\Anaconda\envs\COMP9444\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 [==============================] - 356s 2s/step - loss: 0.6867 - accuracy: 0.6335 - precision: 0.6286 - recall: 0.6765 - val_loss: 0.8316 - val_accuracy: 0.5066 - val_precision: 0.5066 - val_recall: 1.0000
Epoch 2/25
189/189 [==============================] - 329s 2s/step - loss: 0.4944 - accuracy: 0.7644 - precision: 0.7599 - recall: 0.7822 - val_loss: 1.0211 - val_accuracy: 0.5066 - val_precision: 0.5066 - val_recall: 1.0000
Epoch 3/25
189/189 [==============================] - 229s 1s/step - loss: 0.3830 - accuracy: 0.8269 - precision: 0.8314 - recall: 0.8260 - val_loss: 1.7143 - val_accuracy: 0.5066 - val_precision: 0.5066 - val_recall: 1.0000
Epoch 4/25
189/189 [==============================] - 174s 910ms/step - loss: 0.3281 - accuracy: 0.8544 - precision: 0.8587 - recall: 0.8531 - val_loss: 0.4838 - val_accuracy: 0.7815 - val_precision: 0.7341 - val_recall: 0.8915
Epoch 5/25
189/189 [==============================] - 58s 308ms/step - loss: 0.2867 - accuracy: 0.8810 - pre